## traditional graph algorithm

In [ ]:
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix




features = df.drop(['Class'], axis=1).values
labels = df['Class'].values
edge_index = ...  # Construct the edge index based on transaction relationships


x_train, x_test, y_train, y_test, edge_train, edge_test = train_test_split(
    features, labels, edge_index, test_size=0.2, random_state=42
)


G = nx.Graph()
G.add_nodes_from(range(len(x_train)))
G.add_edges_from(edge_train)


# Example 1: Degree Centrality
degree_centrality = nx.degree_centrality(G)

# Example 2: Community Detection
communities = list(nx.community.greedy_modularity_communities(G))


# Example 1: Identify nodes with high degree centrality
high_degree_nodes = [node for node, centrality in degree_centrality.items() if centrality > 0.05]

# Example 2: Identify nodes not belonging to any community
non_community_nodes = [node for node in G.nodes if not any(node in comm for comm in communities)]

# Evaluation
# (Evaluation metrics are not directly applicable for traditional graph algorithms, but you can analyze and visualize the results)

# Note: Adjust the code based on the specific characteristics of your dataset and problem.


## graph neural network

In [ ]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np




features = df.drop(['Class'], axis=1).values
labels = df['Class'].values
edge_index = ...  # construct the edge index based on transaction relationships


x_train, x_test, y_train, y_test, edge_train, edge_test = train_test_split(
    features, labels, edge_index, test_size=0.2, random_state=42
)


train_data = Data(x=torch.tensor(x_train, dtype=torch.float32),
                  edge_index=torch.tensor(edge_train, dtype=torch.long).t().contiguous(),
                  y=torch.tensor(y_train, dtype=torch.float32))
test_data = Data(x=torch.tensor(x_test, dtype=torch.float32),
                 edge_index=torch.tensor(edge_test, dtype=torch.long).t().contiguous(),
                 y=torch.tensor(y_test, dtype=torch.float32))


train_loader = DataLoader([train_data], batch_size=64, shuffle=True)
test_loader = DataLoader([test_data], batch_size=64, shuffle=False)


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(x_train.shape[1], 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.BCEWithLogitsLoss()


model.train()
for epoch in range(50):
    for batch in train_loader:
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, batch.y.unsqueeze(1))
        loss.backward()
        optimizer.step()


model.eval()
y_pred = []
with torch.no_grad():
    for batch in test_loader:
        output = model(batch)
        y_pred.extend(torch.sigmoid(output).cpu().numpy())


y_pred_binary = (np.array(y_pred) > 0.5).astype(int)
print("Classification Report:\n", classification_report(y_test, y_pred_binary))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_binary))


## graph algorithm

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the Credit Card Fraud Detection dataset
data = pd.read_csv("creditcard.csv")

# Standardize the 'Amount' column using StandardScaler
scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))

# Apply PCA to reduce dimensionality (excluding 'Time' and 'Class' columns)
pca = PCA(n_components=2)
data_pca = pd.DataFrame(pca.fit_transform(
    data.drop(['Time', 'Class'], axis=1)), columns=['PCA1', 'PCA2'])

# Create a graph
G = nx.Graph()

# Add nodes with features
for idx, row in data.iterrows():
    G.add_node(idx, features=row.drop(['Time', 'Class']))

# Add edges (connecting nodes with similar features, you can define your own criteria)
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        similarity = sum(data.iloc[i].drop(
            ['Time', 'Class']) == data.iloc[j].drop(['Time', 'Class']))
        if similarity >= 28:  # You can adjust the threshold based on your criteria
            G.add_edge(i, j)

# Visualize the graph (for a small subset of the data)
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(10, 6))
nx.draw(G, pos, node_size=10, node_color='b', with_labels=False)
plt.title("Graph Representation of Credit Card Transactions")
plt.show()
